In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from matplotlib import ticker
from matplotlib.collections import PatchCollection
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Polygon
from sklearn.cluster import DBSCAN
import datetime
import pandas as pd
import time
import shapefile
import os
from os import listdir
from os.path import isfile, isdir, join

from sklearn.cluster import DBSCAN

import warnings
warnings.filterwarnings("ignore")

#Turn off interactive plotting for pyplot
plt.ioff()

In [2]:
only_dirs = [d for d in listdir("Data/") if isdir(join("Data/", d))] # makes a list of all the folders in Data/
dir_count = len(only_dirs)

WWLLN_Track_Path = [""] * dir_count # the paths of all the track files, in the same order as only_dirs
WWLLN_Locations_Path = [""] * dir_count # the paths of all the lightning locations files, in the same order as only_dirs
Cubic_Spline_Path = [""] * dir_count # the paths of all the cubic spline files, in the same order as only_dirs

Image_Path = [""] * dir_count # Base Image paths
Storm_Names = [""] * dir_count # Base storm names

# loop thru all dirs
dir_indx = 0
while dir_indx < dir_count:
    # makes the path for this index's storm
    Image_Path[dir_indx] = join("./Images/", only_dirs[dir_indx])
    
    # generates the name of the storm for images
    Storm_Names[dir_indx] = only_dirs[dir_indx].replace('_', ' ')
    
    # if an image folder |dosen't| exist, create a new one with the path at Image_Path[dir_indx]
    if not os.path.exists(Image_Path[dir_indx]):
        os.mkdir(Image_Path[dir_indx])
    
    Image_Path[dir_indx] = Image_Path[dir_indx] + "/" # add a slash for use in the next block, making the actual img paths
    
    # this gets all the file paths in a given storms' folder
    only_dirs[dir_indx] = join("Data/", only_dirs[dir_indx])
    file_paths = listdir(only_dirs[dir_indx])
    
    # check if the 3 nessisary files exist
    if len(file_paths) != 3:
        only_dirs[dir_indx] = ""
        continue
    
    #print(only_dirs[dir_indx])
    #print(file_paths)
    
    # for every file in file_paths, sort it into its' respective arrays
    file_indx = 0
    while file_indx < 3:
        # creates the files full path
        file_paths[file_indx] = join(only_dirs[dir_indx]+"/", file_paths[file_indx])
        #print(file_paths[file_indx])
        
        # sorts the file path based on the last expected word of a file and it's type
        if file_paths[file_indx].endswith("Trackfile.txt"):
            WWLLN_Track_Path[dir_indx] = file_paths[file_indx]
            #print(WWLLN_Track_Path)
        elif file_paths[file_indx].endswith("Locations.txt"):
            WWLLN_Locations_Path[dir_indx] = file_paths[file_indx]
            #print(WWLLN_Locations_Path)
        elif file_paths[file_indx].endswith("Trackfile.csv"):
            Cubic_Spline_Path[dir_indx] = file_paths[file_indx]
            #print(Cubic_Spline_Path)
        
        file_indx+=1
    
    #print('\n')
    dir_indx+=1

# print for debug
print(Storm_Names)
print(Image_Path)
print(WWLLN_Track_Path)
print(WWLLN_Locations_Path)
print(Cubic_Spline_Path)

['Hurricane Dorian', 'Hurricane Laura']
['./Images/Hurricane_Dorian/', './Images/Hurricane_Laura/']
['Data/Hurricane_Dorian/ATL_19_5_Dorian_Reduced_Trackfile.txt', 'Data/Hurricane_Laura/ATL_20_13_Laura_Reduced_Trackfile.txt']
['Data/Hurricane_Dorian/ATL_19_5_Dorian_WWLLN_Locations.txt', 'Data/Hurricane_Laura/ATL_20_13_Laura_WWLLN_Locations.txt']
['Data/Hurricane_Dorian/ATL_19_5_Dorian_Cubic_Spline_Trackfile.csv', 'Data/Hurricane_Laura/ATL_20_13_Laura_Cubic_Spline_Trackfile.csv']


In [3]:
check_minutes = 30 # minuite var

# for every storm, do the things
dir_indx = 0
while dir_indx < dir_count:
    # if storm is invalid, skip doin the things
    if only_dirs[dir_indx] == "":
        continue
    
    # This loads the WWLLN trackfile Jeremy and Natalia give us
    df = pd.read_csv(WWLLN_Track_Path[dir_indx], header=None, names=["Year","Month","Day","Hour","Lat","Long","Min_Pressure","Max_Winds","Unused"], low_memory=False, sep='\t')
    df = df.drop("Unused", axis=1)
    df['Date'] = df.apply(lambda x: pd.to_datetime(f"{int(x['Year'])}-{int(x['Month'])}-{int(x['Day'])}-{int(x['Hour'])}", format="%Y-%m-%d-%H"), axis=1)

    # This loads the WWLLN lighting location data that Jeremy and Natalia give us
    ln = pd.read_csv(WWLLN_Locations_Path[dir_indx], header=None, names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"], low_memory=False, sep=' ')
    ln['Date'] = ln.apply(lambda x: pd.to_datetime(f"{int(x['Year'])}-{int(x['Month'])}-{int(x['Day'])}-{int(x['Hour'])}-{int(x['Min'])}-{x['Sec']}", format="%Y-%m-%d-%H-%M-%S.%f"), axis=1)

    # This opens the cubic spline trackfiles that Ben creates
    storm_center = pd.read_csv(Cubic_Spline_Path[dir_indx])
    storm_center['Date'] = dorian_center['Date'].apply(pd.to_datetime)

    # Genereates the list of dates from the WWLLN trackfile of a (check_minutes) range
    dates = pd.date_range(df['Date'][0], df['Date'][len(df)-1], freq=str(check_minutes)+"T")

    buffer = 10 # affects map zooooooooooooom
    shp_file = "World_Countries__Generalized_-shp/World_Countries__Generalized_" # Map file ???????
    cmap = 'gist_ncar' # dunno WHAT this is

    # Counter and iteration
    for index, date in enumerate(dates): # this loop makes one image for every (check_minutes) amount of time
        starttime = time.perf_counter() # speeed timer start

        # Select just the lightning locations that are within (check_minutes) minutes
        subset = ln[(ln['Date'] >= date) & (ln['Date'] < date + datetime.timedelta(minutes=check_minutes))]
        center = storm_center[storm_center['Date'] == date] # storm center at this time frame
        
        # if lightning exists, do dbscan on it
        if len(subset != 0):
            dbscan = DBSCAN(eps=.2, min_samples=2)
            subset['Labels'] = dbscan.fit_predict(subset[['Long','Lat']])

        # Setup the figure
        fig, ax = plt.subplots(figsize=(15,15))
        
        # this is for the coastline
        # This reads the shapefile, extracts each shape, creates a polygon, and adds it to the list
        sf = shapefile.Reader(shp_file)
        shapes = sf.shapes()
        Nshp = len(shapes)
        ptchs = []
        for nshp in range(Nshp):
            pts = np.array(shapes[nshp].points)
            prt = shapes[nshp].parts
            par = list(prt) + [pts.shape[0]]
            for pij in range(len(prt)):# <--------------------------------------------------------------------------------------------------------------------------- FIX ME (im slow) use while loops, never range()!
                ptchs.append(Polygon(pts[par[pij]:par[pij+1]]))
        # Every every polygon from the shapefile to figure
        ax.add_collection(PatchCollection(ptchs, facecolor= '#838688', edgecolor='k', linewidths=1., zorder=2))

        # This adds the water color
        ax.add_patch(mpl.patches.Rectangle((-180,-89),360,180,color='#a6cae0'))

        # This connects Antarcitca to the bottom of the image
        ax.add_patch(mpl.patches.Rectangle((-179.9,-89.9),360,2,color='#838688',zorder=3))

        # These are more general image settings
        plt.title(Storm_Names[dir_indx]+' '+str(date))

        #edges = (-180,180,-90,90) # Left, Right, Bottom, Top | Set this if you want to change the map scale
        edges = (center['Long'].values[0] - buffer, center['Long'].values[0] + buffer, center['Lat'].values[0] - buffer, center['Lat'].values[0] + buffer) # <--------------------------This affects map zoooooooooom

        # setting up img stuff, prob shouldn't be touched
        xlim = np.append(np.arange(edges[0], edges[1], step=2.5), edges[1])
        ylim = np.append(np.arange(edges[2], edges[3], step=2.5), edges[3])
        ax.set_xticks(xlim)
        ax.set_yticks(ylim)
        ax.set_xlim(edges[0], edges[1])
        ax.set_ylim(edges[2], edges[3])
        ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
        ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))

        # if lightning exists, put it in the image
        if len(subset) != 0:
            plt.scatter(subset['Long'], subset['Lat'], c=subset['Labels'], cmap=cmap, zorder=5) # plots the dbscan data
        
        #plt.scatter(subset['Long'],subset['Lat'],s=100,c="yellow",edgecolors='black',zorder=5) # <- old plot func for non-grouped data
        plt.scatter(center['Long'], center['Lat'], s=140, c="red", edgecolors='black' ,zorder=4) # plots the center of the storm

        # save and close the image
        plt.savefig(Image_Path[dir_indx] + str(index), bbox_inches='tight', pad_inches=.4)
        plt.close('all')

        # Shows roughly how long we have left
        taken = time.perf_counter() - starttime
        print(f"Percent Done: {(index + 1) / len(dates) * 100 : .2f}%\tTime taken: {taken : .2f} seconds\tEst time remaining: {datetime.timedelta(seconds=(len(dates) - (index + 1)) * taken)} {len(subset)}", end='\r')
    dir_indx+=1